#### This notebook walks through the steps in creating an aggregrated dataset over the three categories (Hobbies,Foods and Household)
- to run this notebook please have the datasets, "sales_train_evaluation_csv" and "calendar_csv" downloaded
- the desired aggregated dataset is "cat_TS" which will exported as csv file in the name "category_data", and will be used in developing the ML and statistical models

In [2]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from datetime import datetime
from pyspark.sql.functions import udf
from pyspark.sql.types import DateType
from pyspark.sql import *
from pyspark.sql.window import Window
from pyspark.sql import functions as f
from pyspark.sql.functions import col, first
from pyspark.sql.types import FloatType, StringType
from pyspark.sql.functions import monotonically_increasing_id
from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_squared_error
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from pylab import rcParams
rcParams['figure.figsize'] = 20, 8

In [3]:
unit_sales = spark.table("sales_train_evaluation_csv")
display(unit_sales)

id item_id dept_id cat_id store_id state_id d_1 d_2 d_3 d_4 d_5 d_6 d_7 d_8 d_9 d_10 d_11 d_12 d_13 d_14 d_15 d_16 d_17 d_18 d_19 d_20 d_21 d_22 d_23 d_24 d_25 d_26 d_27 d_28 d_29 d_30 d_31 d_32 d_33 d_34 d_35 d_36 d_37 d_38 d_39 d_40 d_41 d_42 d_43 d_44 d_45 d_46 d_47 d_48 d_49 d_50 d_51 d_52 d_53 d_54 d_55 d_56 d_57 d_58 d_59 d_60 d_61 d_62 d_63 d_64 d_65 d_66 d_67 d_68 d_69 d_70 d_71 d_72 d_73 d_74 d_75 d_76 d_77 d_78 d_79 d_80 d_81 d_82 d_83 d_84 d_85 d_86 d_87 d_88 d_89 d_90 d_91 d_92 d_93 d_94 d_95 d_96 d_97 d_98 d_99 d_100 d_101 d_102 d_103 d_104 d_105 d_106 d_107 d_108 d_109 d_110 d_111 d_112 d_113 d_114 d_115 d_116 d_117 d_118 d_119 d_120 d_121 d_122 d_123 d_124 d_125 d_126 d_127 d_128 d_129 d_130 d_131 d_132 d_133 d_134 d_135 d_136 d_137 d_138 d_139 d_140 d_141 d_142 d_143 d_144 d_145 d_146 d_147 d_148 d_149 d_150 d_151 d_152 d_153 d_154 d_155 d_156 d_157 d_158 d_159 d_160 d_161 d_162 d_163 d_164 d_165 d_166 d_167 d_168 d_169 d_170 d_171 d_172 d_173 d_174 d_175 d_176 d_177 d_178 d_179 d_180 d_181 d_182 d_183 d_184 d_185 d_186 d_187 d_188 d_189 d_190 d_191 d_192 d_193 d_194 d_195 d_196 d_197 d_198 d_199 d_200 d_201 d_202 d_203 d_204 d_205 d_206 d_207 d_208 d_209 d_210 d_211 d_212 d_213 d_214 d_215 d_216 d_217 d_218 d_219 d_220 d_221 d_222 d_223 d_224 d_225 d_226 d_227 d_228 d_229 d_230 d_231 d_232 d_233 d_234 d_235 d_236 d_237 d_238 d_239 d_240 d_241 d_242 d_243 d_244 d_245 d_246 d_247 d_248 d_249 d_250 d_251 d_252 d_253 d_254 d_255 d_256 d_257 d_258 d_259 d_260 d_261 d_262 d_263 d_264 d_265 d_266 d_267 d_268 d_269 d_270 d_271 d_272 d_273 d_274 d_275 d_276 d_277 d_278 d_279 d_280 d_281 d_282 d_283 d_284 d_285 d_286 d_287 d_288 d_289 d_290 d_291 d_292 d_293 d_294 d_295 d_296 d_297 d_298 d_299 d_300 d_301 d_302 d_303 d_304 d_305 d_306 d_307 d_308 d_309 d_310 d_311 d_312 d_313 d_314 d_315 d_316 d_317 d_318 d_319 d_320 d_321 d_322 d_323 d_324 d_325 d_326 d_327 d_328 d_329 d_330 d_331 d_332 d_333 d_334 d_335 d_336 d_337 d_338 d_339 d_340 d_341 d_342 d_343 d_344 d_345 d_346 d_347 d_348 d_349 d_350 d_351 d_352 d_353 d_354 d_355 d_356 d_357 d_358 d_359 d_360 d_361 d_362 d_363 d_364 d_365 d_366 d_367 d_368 d_369 d_370 d_371 d_372 d_373 d_374 d_375 d_376 d_377 d_378 d_379 d_380 d_381 d_382 d_383 d_384 d_385 d_386 d_387 d_388 d_389 d_390 d_391 d_392 d_393 d_394 d_395 d_396 d_397 d_398 d_399 d_400 d_401 d_402 d_403 d_404 d_405 d_406 d_407 d_408 d_409 d_410 d_411 d_412 d_413 d_414 d_415 d_416 d_417 d_418 d_419 d_420 d_421 d_422 d_423 d_424 d_425 d_426 d_427 d_428 d_429 d_430 d_431 d_432 d_433 d_434 d_435 d_436 d_437 d_438 d_439 d_440 d_441 d_442 d_443 d_444 d_445 d_446 d_447 d_448 d_449 d_450 d_451 d_452 d_453 d_454 d_455 d_456 d_457 d_458 d_459 d_460 d_461 d_462 d_463 d_464 d_465 d_466 d_467 d_468 d_469 d_470 d_471 d_472 d_473 d_474 d_475 d_476 d_477 d_478 d_479 d_480 d_481 d_482 d_483 d_484 d_485 d_486 d_487 d_488 d_489 d_490 d_491 d_492 d_493 d_494 d_495 d_496 d_497 d_498 d_499 d_500 d_501 d_502 d_503 d_504 d_505 d_506 d_507 d_508 d_509 d_510 d_511 d_512 d_513 d_514 d_515 d_516 d_517 d_518 d_519 d_520 d_521 d_522 d_523 d_524 d_525 d_526 d_527 d_528 d_529 d_530 d_531 d_532 d_533 d_534 d_535 d_536 d_537 d_538 d_539 d_540 d_541 d_542 d_543 d_544 d_545 d_546 d_547 d_548 d_549 d_550 d_551 d_552 d_553 d_554 d_555 d_556 d_557 d_558 d_559 d_560 d_561 d_562 d_563 d_564 d_565 d_566 d_567 d_568 d_569 d_570 d_571 d_572 d_573 d_574 d_575 d_576 d_577 d_578 d_579 d_580 d_581 d_582 d_583 d_584 d_585 d_586 d_587 d_588 d_589 d_590 d_591 d_592 d_593 d_594 d_595 d_596 d_597 d_598 d_599 d_600 d_601 d_602 d_603 d_604 d_605 d_606 d_607 d_608 d_609 d_610 d_611 d_612 d_613 d_614 d_615 d_616 d_617 d_618 d_619 d_620 d_621 d_622 d_623 d_624 d_625 d_626 d_627 d_628 d_629 d_630 d_631 d_632 d_633 d_634 d_635 d_636 d_637 d_638 d_639 d_640 d_641 d_642 d_643 d_644 d_645 d_646 d_647 d_648 d_649 d_650 d_651 d_652 d_653 d_654 d_655 d_656 d_657 d_658 d_659 d_660 d_661 d_662 d_663 d_664 d_665 d_666 d_667 d_668 d_669 d_670 d_671 d_672 d_673 d_674 d_675 d_676 d_677 d_

In [4]:
from pyspark.sql.functions import split
cal = spark.table("calendar_csv")
cal_df = cal.withColumn("d", split("d", "_").getItem(1))

change_date =  udf (lambda x: datetime.strptime(x, '%Y-%m-%d'), DateType())

cal_df = cal_df.withColumn('Date', change_date(col('date')))
cal_df.drop("date")


cal_df.show()

+----------+--------+---------+----+-----+----+---+-------------+------------+------------+------------+-------+-------+-------+
 Date|wm_yr_wk| weekday|wday|month|year| d| event_name_1|event_type_1|event_name_2|event_type_2|snap_CA|snap_TX|snap_WI|
+----------+--------+---------+----+-----+----+---+-------------+------------+------------+------------+-------+-------+-------+
2011-01-29| 11101| Saturday| 1| 1|2011| 1| null| null| null| null| 0| 0| 0|
2011-01-30| 11101| Sunday| 2| 1|2011| 2| null| null| null| null| 0| 0| 0|
2011-01-31| 11101| Monday| 3| 1|2011| 3| null| null| null| null| 0| 0| 0|
2011-02-01| 11101| Tuesday| 4| 2|2011| 4| null| null| null| null| 1| 1| 0|
2011-02-02| 11101|Wednesday| 5| 2|2011| 5| null| null| null| null| 1| 0| 1|
2011-02-03| 11101| Thursday| 6| 2|2011| 6| null| null| null| null| 1| 1| 1|
2011-02-04| 11101| Friday| 7| 2|2011| 7| null| null| null| null| 1| 0| 0|
2011-02-05| 11102| Saturday| 1| 2|2011| 8| null| null| null| null| 1| 1| 1|
2011-02-06| 11102| Sunday| 2| 2|2011| 9| SuperBowl| Sporting| null| null| 1| 1| 1|
2011-02-07| 11102| Monday| 3| 2|2011| 10| null| null| null| null| 1| 1| 0|
2011-02-08| 11102| Tuesday| 4| 2|2011| 11| null| null| null| null| 1| 0| 1|
2011-02-09| 11102|Wednesday| 5| 2|2011| 12| null| null| null| null| 1| 1| 1|
2011-02-10| 11102| Thursday| 6| 2|2011| 13| null| null| null| null| 1| 0| 0|
2011-02-11| 11102| Friday| 7| 2|2011| 14| null| null| null| null| 0| 1| 1|
2011-02-12| 11103| Saturday| 1| 2|2011| 15| null| null| null| null| 0| 1| 1|
2011-02-13| 11103| Sunday| 2| 2|2011| 16| null| null| null| null| 0| 1| 0|
2011-02-14| 11103| Monday| 3| 2|2011| 17|ValentinesDay| Cultural| null| null| 0| 0| 1|
2011-02-15| 11103| Tuesday| 4| 2|2011| 18| null| null| null| null| 0| 1| 1|
2011-02-16| 11103|Wednesday| 5| 2|2011| 19| null| null| null| null| 0| 0| 0|
2011-02-17| 11103| Thursday| 6| 2|2011| 20| null| null| null| null| 0| 0| 0|
+----------+--------+---------+----+-----+----+---+-------------+------------+------------+------------+-------+-------+-------+
only showing top 20 rows

#####DataFrame for Time Series Agg over categories:

In [6]:
#summing up data across rows for d1,d2...d1941 for each category
Hob = unit_sales.filter(col("cat_id")=="HOBBIES")
H = [f.sum(x).alias(str(x)) for x in Hob.columns[6:]]
H1 = Hob.select(H).collect()[0]
Hob_units = spark.createDataFrame(H1, FloatType())
Hob_units = Hob_units.withColumnRenamed("value","Hobbies")

Food = unit_sales.filter(col("cat_id")=="FOODS")
F = [f.sum(x).alias(str(x)) for x in Food.columns[6:]]
F1 = Food.select(F).collect()[0]
Food_units = spark.createDataFrame(F1, FloatType())
Food_units = Food_units.withColumnRenamed("value","Foods")

House = unit_sales.filter(col("cat_id")=="HOUSEHOLD")
Ho = [f.sum(x).alias(str(x)) for x in House.columns[6:]]
Ho1 = House.select(Ho).collect()[0]
House_units = spark.createDataFrame(Ho1, FloatType())
House_units = House_units.withColumnRenamed("value","HouseHold")

In [7]:
#joining the above columns into a dataframe
df1 = Hob_units.withColumn("id", monotonically_increasing_id())
df2 = Food_units.withColumn("id", monotonically_increasing_id())
df3 = House_units.withColumn("id", monotonically_increasing_id())
cat_df = df1.join(df2,["id"]).join(df3,["id"]).sort("id")

#Dataframe for aggregration over catergory
window = Window.orderBy(f.col('id'))
cat_df = cat_df.withColumn('day', f.row_number().over(window)).drop("id")
display(cat_df)

Hobbies,Foods,HouseHold,day
3764.0,23178.0,5689.0,1
3357.0,22758.0,5634.0,2
2682.0,17174.0,3927.0,3
2669.0,18878.0,3865.0,4
1814.0,14603.0,2729.0,5
3220.0,22093.0,3898.0,6
2944.0,20490.0,4576.0,7
3986.0,27751.0,6195.0,8
2899.0,24862.0,4975.0,9
2615.0,18901.0,4056.0,10


In [8]:
#this is the desired dataset "cat_TS" 
#this is downloaded as csv and saved under the name "category_data"

#merging cat_df with the calendar_df
cat_TS = cat_df.join(cal_df,cat_df.day == cal_df.d).select("Date","d","wm_yr_wk","weekday","month","year","Hobbies","Foods","HouseHold","event_name_1","event_type_1","event_name_2","event_type_2")
cat_TS.show()

+----------+---+--------+---------+-----+----+-------+-------+---------+-------------+------------+------------+------------+
 Date| d|wm_yr_wk| weekday|month|year|Hobbies| Foods|HouseHold| event_name_1|event_type_1|event_name_2|event_type_2|
+----------+---+--------+---------+-----+----+-------+-------+---------+-------------+------------+------------+------------+
2011-01-29| 1| 11101| Saturday| 1|2011| 3764.0|23178.0| 5689.0| null| null| null| null|
2011-01-30| 2| 11101| Sunday| 1|2011| 3357.0|22758.0| 5634.0| null| null| null| null|
2011-01-31| 3| 11101| Monday| 1|2011| 2682.0|17174.0| 3927.0| null| null| null| null|
2011-02-01| 4| 11101| Tuesday| 2|2011| 2669.0|18878.0| 3865.0| null| null| null| null|
2011-02-02| 5| 11101|Wednesday| 2|2011| 1814.0|14603.0| 2729.0| null| null| null| null|
2011-02-03| 6| 11101| Thursday| 2|2011| 3220.0|22093.0| 3898.0| null| null| null| null|
2011-02-04| 7| 11101| Friday| 2|2011| 2944.0|20490.0| 4576.0| null| null| null| null|
2011-02-05| 8| 11102| Saturday| 2|2011| 3986.0|27751.0| 6195.0| null| null| null| null|
2011-02-06| 9| 11102| Sunday| 2|2011| 2899.0|24862.0| 4975.0| SuperBowl| Sporting| null| null|
2011-02-07| 10| 11102| Monday| 2|2011| 2615.0|18901.0| 4056.0| null| null| null| null|
2011-02-08| 11| 11102| Tuesday| 2|2011| 2547.0|16963.0| 3561.0| null| null| null| null|
2011-02-09| 12| 11102|Wednesday| 2|2011| 2457.0|17655.0| 3546.0| null| null| null| null|
2011-02-10| 13| 11102| Thursday| 2|2011| 2700.0|17340.0| 3716.0| null| null| null| null|
2011-02-11| 14| 11102| Friday| 2|2011| 2822.0|19038.0| 4585.0| null| null| null| null|
2011-02-12| 15| 11103| Saturday| 2|2011| 3699.0|25187.0| 5947.0| null| null| null| null|
2011-02-13| 16| 11103| Sunday| 2|2011| 3417.0|26656.0| 6307.0| null| null| null| null|
2011-02-14| 17| 11103| Monday| 2|2011| 2233.0|16015.0| 3556.0|ValentinesDay| Cultural| null| null|
2011-02-15| 18| 11103| Tuesday| 2|2011| 2554.0|17425.0| 4091.0| null| null| null| null|
2011-02-16| 19| 11103|Wednesday| 2|2011| 2170.0|15351.0| 3922.0| null| null| null| null|
2011-02-17| 20| 11103| Thursday| 2|2011| 2049.0|14290.0| 3979.0| null| null| null| null|
+----------+---+--------+---------+-----+----+-------+-------+---------+-------------+------------+------------+------------+
only showing top 20 rows